In [7]:
#basic data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

# web scraping and nlp imports
import nltk
import re
import os
import json
from requests import get

# Soupy Soup
from bs4 import BeautifulSoup

[‘LEANDER H S’,
 ‘TASCOSA H S’,
 ‘CEDAR RIDGE H S’,
 ‘HUNTSVILLE H S’,
 ‘SMITHSON VALLEY H S’,
 ‘EL CAMPO H S’,
 ‘DEVINE H S’,
 ‘TOMBALL H S’,
 ‘REAGAN COUNTY H S’,
 ‘ALTER SCH’,
 ‘CANEY CREEK H S’,
 ‘FRUITVALE H S’,
 ‘NEW WAVERLY H S’,
 ‘LBJ ECHS’,
 ‘HIGHLAND PARK H S’,
 ‘WINTERS H S’,
 ‘BROWNWOOD ACCELERATED H S’,
 ‘PILOT POINT H S’,
 ‘NEWTON H S’,
 ‘VEGA H S’,
 ‘DICKINSON H S’,
 ‘HENRY FORD ACADEMY ALAMEDA SCHOOL FOR ART + DESIGN’,
 ‘LUDI PENA MARTIN ACCELERATED EDUCATION CENTER’,
 ‘ROWE H S’]

In [2]:
# URL for STAAR score page
url = 'https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=0&lev=C&id=246913001&prgopt=reports%2Ftapr%2Fperformance.sas'


In [3]:
# ping STAAR page
response = get(url)
response

<Response [200]>

In [4]:
response.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html>\n<head>\n<title>TPRS</title><meta name=\'keywords\' contents = \'5042\'> \n<meta name="Generator" content="SAS Software Version 9.4, see www.sas.com">\n<meta http-equiv="Content-type" content="text/html; charset=iso-8859-1">\n<title>SAS Output</title>\n<style type="text/css">\n<!--\n.accessiblecaption\n{\n  background-color: transparent;\n  border: 0px solid #000000;\n  border-spacing: 0px;\n  color: #000000;\n  font-family: Arial, \'Albany AMT\', sans-serif;\n  font-size: 12pt;\n  font-style: normal;\n  font-weight: bold;\n  margin: 0px;\n  padding: 0px;\n}\n.aligncontents\n{\n  background-color: transparent;\n  border: 0px solid #000000;\n  border-spacing: 0px;\n  color: #000000;\n  font-family: \'Trebuchet MS\', Helvetica, Arial, \'Albany AMT\', sans-serif;\n  font-size: 12pt;\n  font-style: normal;\n  font-weight: normal;\n  margin: 0px;\n  padding: 5px;\n  text-align: left;\n  vertical-align: middle;\n}\n.bat

In [5]:
# Beautify the text
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<title>TPRS</title><meta contents="5042" name="keywords"/>
<meta content="SAS Software Version 9.4, see www.sas.com" name="Generator"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<title>SAS Output</title>
<style type="text/css">
<!--
.accessiblecaption
{
  background-color: transparent;
  border: 0px solid #000000;
  border-spacing: 0px;
  color: #000000;
  font-family: Arial, 'Albany AMT', sans-serif;
  font-size: 12pt;
  font-style: normal;
  font-weight: bold;
  margin: 0px;
  padding: 0px;
}
.aligncontents
{
  background-color: transparent;
  border: 0px solid #000000;
  border-spacing: 0px;
  color: #000000;
  font-family: 'Trebuchet MS', Helvetica, Arial, 'Albany AMT', sans-serif;
  font-size: 12pt;
  font-style: normal;
  font-weight: normal;
  margin: 0px;
  padding: 5px;
  text-align: left;
  vertical-align: middle;
}
.batch
{
  background-color: #FFFFFF;
  border: 1px soli

In [6]:
# Narrow down results
x = soup.find_all('tr')[20]
x

<tr>
<th class="l t data" id="rh2b1c19" rowspan="2">At Approaches Grade Level or Above                       </th>
<th class="c t data" id="rh2b1c20">     2022</th>
<td class="r t data">    76%   </td>
<td class="r t data">    81%   </td>
<td class="r t data" style=" font-weight: bold;">    66%   </td>
<td class="r t data">    67%   </td>
<td class="r t data">    59%   </td>
<td class="r t data">    68%   </td>
<td class="r t data">*         </td>
<td class="r t data">    88%   </td>
<td class="r t data">-         </td>
<td class="r t data">    83%   </td>
<td class="r t data">    44%   </td>
<td class="r t data">    40%   </td>
<td class="r t data">    66%   </td>
<td class="r t data">    65%   </td>
<td class="r t data">    55%   </td>
<td class="r t data">    50%   </td>
</tr>

### Algebra Score

In [7]:
# Hone in on target text. Algebra score
x.find_all('td')[2].text

'    66%   '

In [8]:
yy = soup.find_all('td', class_='c systemtitle3')[0].text
yy

'LEANDER H S (246913001) - LEANDER ISD - WILLIAMSON COUNTY'

In [9]:
# Econ Dis
x.find_all('td')[14].text

'    55%   '

In [10]:
# Eb El
x.find_all('td')[15].text

'    50%   '

### Webs Scrape

In [11]:
# URL Comparison
# Difference: Campus Number
tascosa_url = '''
            https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
            0&lev=C&id=188901005&prgopt=reports%2Ftapr%2Fperformance.sas
            '''
leander_url =   '''
            https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
            0&lev=C&id=246913001&prgopt=reports%2Ftapr%2Fperformance.sas
            
            '''

In [12]:
# Campus Numbers

leander_cn = '246913001'
tascosa_cn = '188901005'
cedar_ridge_cn = '246909008'
huntsville_cn = '236902002'
smithson_valley_cn = '046902002'
el_campo_cn = '241903001'
devine_cn = '163901001'
tomball_cn = '101921001'
reagan_county_cn = '192901001'

In [13]:
campus_names = ['leander','tascosa','cedar_ridge','huntsville','smithson_valley','el_campo','devine','tomball',
              'reagan_county']

In [14]:
# Campus variable list

campus_number_list = [leander_cn,tascosa_cn,cedar_ridge_cn,huntsville_cn,
                     smithson_valley_cn,el_campo_cn,devine_cn,tomball_cn,reagan_county_cn]

In [15]:
test_list = []
test_list

[]

In [16]:
# STAAR Score Loop

test_list = []

for cn in campus_number_list:
    
    url = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
        0&lev=C&id={cn}&prgopt=reports%2Ftapr%2Fperformance.sas'''
            
            
    
    # Ping URL
    response = get(url)
    print(response)
    
    # Soupy soup
    soup = BeautifulSoup(response.content, 'html.parser')

    x = soup.find_all('tr')[20]
    
    # Hone in on target text. Algebra score
    xx = x.find_all('td')[2].text
    
    test_list.append(xx)
    
    

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [17]:
test_list

['    66%   ',
 '    64%   ',
 '    62%   ',
 '    41%   ',
 '    84%   ',
 '    63%   ',
 '    82%   ',
 '    86%   ',
 '    75%   ']

In [18]:
def get_staar_scores(): 
    '''
    Web scaper for STAAR scores.
    '''
    staar_list = []
    name_list = []
    for cn in campus_number_list:

        url = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
            0&lev=C&id={cn}&prgopt=reports%2Ftapr%2Fperformance.sas'''



        # Ping URL
        response = get(url)
        print(response)

        # Soupy soup
        soup = BeautifulSoup(response.content, 'html.parser')

        x = soup.find_all('tr')[20]

        # Hone in on target text. Algebra score
        xx = x.find_all('td')[2].text

        staar_list.append(xx)
    
    for n in campus_names:
        
        name_list.append(n)
            
    df = pd.DataFrame({'school':name_list ,'algebra':staar_list})

    return df


In [19]:
get_staar_scores()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


,school,algebra
0,leander,66%
1,tascosa,64%
2,cedar_ridge,62%
3,huntsville,41%
4,smithson_valley,84%
5,el_campo,63%
6,devine,82%
7,tomball,86%
8,reagan_county,75%


In [20]:
# JSON file of all Texas schools
schools_df = pd.read_json('schools.json')

In [21]:
# Columns
schools_df.columns

Index(['id', 'district_id', 'district_name', 'paired_id', 'region_id',
       'region', 'county_id', 'county', 'entity_cd', 'entity',
       'entity_type_cd', 'entity_type', 'campus_type_cd', 'campus_type',
       'alt_standards', 'aea_bonus', 'online_school', 'type_cat_cd',
       'type_cat', 'enrollment_type_cd', 'enrollment_type', 'schools',
       'enrollment', 'low_grade_cd', 'low_grade', 'high_grade_cd',
       'high_grade', 'designation_cd', 'designation', 'name', 'search_name',
       'phone', 'email', 'website', 'principal', 'superintendent', 'address',
       'street_address', 'city', 'state', 'zip_5', 'latitude', 'longitude',
       'is_new', 'peg', 'rating_cd', 'rating', 'appeal', 'mult_year', 'score',
       'd1s', 'd1g', 'd1as', 'd1ag', 'd1bs', 'd1bg', 'd1cs', 'd1cg', 'd2s',
       'd2g', 'd2as', 'd2ag', 'd2bs', 'd2bg', 'd3s', 'd3g', 'has_finance',
       'sat_act_cd', 'sat_act', 'gaps', 'd3_academic', 'd3_growth', 'd3_grad',
       'd3_elp', 'd3_student', 'd3_quality', '

In [22]:
schools_df[['id', 'district_id', 'district_name', 'paired_id', 'region_id',
       'region','entity','entity_type_cd', 'entity_type', 'campus_type_cd', 'campus_type',
       'alt_standards', 'aea_bonus', 'online_school',
       'type_cat',
       'enrollment', 'low_grade_cd', 'low_grade', 'high_grade_cd',
       'high_grade']].head()

,id,district_id,district_name,paired_id,region_id,region,entity,entity_type_cd,entity_type,campus_type_cd,campus_type,alt_standards,aea_bonus,online_school,type_cat,enrollment,low_grade_cd,low_grade,high_grade_cd,high_grade
0,1902001,1902,Cayuga ISD,,7,Region 07: Kilgore,Campus,T,Traditional,S,High School,No,No,No,,169,09,Grade 9,12,Grade 12
1,1902041,1902,Cayuga ISD,,7,Region 07: Kilgore,Campus,T,Traditional,M,Middle / Jr. High,No,No,No,,159,06,Grade 6,08,Grade 8
2,1902103,1902,Cayuga ISD,,7,Region 07: Kilgore,Campus,T,Traditional,E,Elementary,No,No,No,,246,KG,Kindergarten,05,Grade 5
3,1903001,1903,Elkhart ISD,,7,Region 07: Kilgore,Campus,T,Traditional,S,High School,No,No,No,,307,09,Grade 9,12,Grade 12
4,1903041,1903,Elkhart ISD,,7,Region 07: Kilgore,Campus,T,Traditional,M,Middle / Jr. High,No,No,No,,268,06,Grade 6,08,Grade 8


In [23]:
high_schools = schools_df[schools_df['campus_type'] == 'High School']
high_schools.head()

,id,district_id,district_name,paired_id,region_id,region,county_id,county,entity_cd,entity,...,gaps,d3_academic,d3_growth,d3_grad,d3_elp,d3_student,d3_quality,distinctions,student_achievement,las_flag
0,1902001,1902,Cayuga ISD,,7,Region 07: Kilgore,1,Anderson,C,Campus,...,"[aa, gr, sq]",100.0,NaN,100.0,NaN,NaN,100.0,"[Y, Y, Y, N, Y, Y, Y]","[Y, Y, Y]",N
3,1903001,1903,Elkhart ISD,,7,Region 07: Kilgore,1,Anderson,C,Campus,...,"[aa, gr, sq]",50.0,NaN,100.0,NaN,NaN,100.0,"[Y, N, Y, Y, N, N, N]","[Y, Y, Y]",N
7,1904001,1904,Frankston ISD,,7,Region 07: Kilgore,1,Anderson,C,Campus,...,"[aa, gr, sq]",100.0,NaN,100.0,NaN,NaN,100.0,"[Y, Y, N, N, Y, Y, Y]","[Y, Y, Y]",N
10,1906002,1906,Neches ISD,,7,Region 07: Kilgore,1,Anderson,C,Campus,...,,NaN,NaN,NaN,NaN,NaN,NaN,"[Y, Y, N, N, Y, Y, X]","[Y, Y, Y]",N
12,1907001,1907,Palestine ISD,,7,Region 07: Kilgore,1,Anderson,C,Campus,...,"[aa, gr, el, sq]",78.0,NaN,100.0,0.0,NaN,100.0,"[Y, Y, Y, N, Y, Y, Y]","[Y, Y, Y]",N


In [24]:
high_schools.shape

(1784, 79)

In [25]:
test_list = []

for i in schools_df.id:
    
    test_list.append(i)

In [26]:
test_list[:10]

[1902001,
 1902041,
 1902103,
 1903001,
 1903041,
 1903101,
 1903102,
 1904001,
 1904041,
 1904102]

In [27]:
def get_staar_scores(): 
    '''
    Web scaper for STAAR scores.
    '''
    staar_list = []
    name_list = []
    for cn in campus_number_list:

        url = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
            0&lev=C&id={cn}&prgopt=reports%2Ftapr%2Fperformance.sas'''



        # Ping URL
        response = get(url)
        print(response)

        # Soupy soup
        soup = BeautifulSoup(response.content, 'html.parser')

        x = soup.find_all('tr')[20]

        # Hone in on target text. Algebra score
        xx = x.find_all('td')[2].text

        staar_list.append(xx)
    
    for n in campus_names:
        
        name_list.append(n)
            
    df = pd.DataFrame({'school':name_list ,'algebra':staar_list})

    return df


In [28]:
get_staar_scores()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


,school,algebra
0,leander,66%
1,tascosa,64%
2,cedar_ridge,62%
3,huntsville,41%
4,smithson_valley,84%
5,el_campo,63%
6,devine,82%
7,tomball,86%
8,reagan_county,75%


In [4]:
def get_school_ids():
    
    
    # JSON file of all Texas schools
    schools_df = pd.read_json('schools.json')
    
    # Filter for High Schools
    schools_df = schools_df[schools_df['campus_type'] == 'High School']
    
    id_number = []

    for i in schools_df.id:

        id_number.append(i)
    
    return id_number
    

In [30]:
all_the_school = get_school_ids()

In [31]:
len(all_the_school)

1784

In [32]:
all_the_school[:20]

[1902001,
 1903001,
 1904001,
 1906002,
 1907001,
 1908002,
 1909001,
 2901001,
 2901002,
 3801001,
 3801002,
 3902001,
 3902004,
 3903001,
 3903004,
 3903201,
 3904001,
 3904005,
 3905001,
 3905004]

In [48]:
def get_staar(): 
    '''
    Web scaper.
    '''
    
    algebra_list = []
    id_number_list = []
    
    # Get all High School IDs
    id_number = get_school_ids()
    
    id_number = id_number[1690:-1]
    
    # Loop for Algebra scores
    for number in id_number:
        
      # URL
        url_staar = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
                0&lev=C&id={number}&prgopt=reports%2Ftapr%2Fperformance.sas'''  
            
        try:
              
            # Ping URLs
            response_staar = get(url_staar)
            

            # Soupy soups
            soup_staar = BeautifulSoup(response_staar.content, 'html.parser')
            
            
            # STAAR: Algebra score
            staar = soup_staar.find_all('tr')[20]
            algebra = staar.find_all('td')[2].text
            

            # Append List
            id_number_list.append(cn)
            algebra_list.append(algebra)
              
        except:
            continue
    
    df = pd.DataFrame({'id':id_number_list ,'algebra':algebra_list})
    return df


In [49]:
staar_score = get_staar()
staar_score.head()

['    76%   ', '-         ', '    63%   ', '    88%   ', '    63%   ', '    60%   ', '    67%   ', '    63%   ', '    72%   ', '    17%   ', '    96%   ', '    86%   ', '    71%   ', '    75%   ', '    55%   ', '*         ', '    74%   ', '-         ', '-         ', '    57%   ', '    79%   ', '    88%   ', '*         ', '    89%   ', '    80%   ', '    50%   ', '    93%   ', '-         ', '    53%   ', '    32%   ', '-         ', '-         ', '    59%   ', '    65%   ', '-         ', '-         ', '-         ', '    80%   ', '    73%   ', '    62%   ', '    52%   ', '    62%   ', '    27%   ', '-         ', '    81%   ', '    58%   ', '    35%   ', '    85%   ', '    66%   ', '    82%   ', '    76%   ', '    73%   ', '    79%   ', '    64%   ', '*         ', '    79%   ', '    77%   ', '    95%   ', '    86%   ', '    77%   ', '    73%   ', '    73%   ', '    78%   ', '    64%   ', '    68%   ', '    67%   ', '    91%   ', '    79%   ', '    84%   ', '    72%   ', '    81%   ', '    

,id,algebra
0,192901001,76%
1,192901001,-
2,192901001,63%
3,192901001,88%
4,192901001,63%


In [19]:
def get_staar_students_staff_finance(): 
    '''
    Web scraper.
    '''
    
    algebra_list = []
    ebel_list = []
    econ_dis_list = []
    id_number_list = []
    salary_list = []
    masters_list = []
    doct_list = []
    beginning_teach_list = [] 
    exp_5_list = []
    exp_10_list = []
    exp_1120_list = []
    exp_2130_list = []
    exp_plus_list = []
    ratio_list = [] 
    extra_fund_list = []
    all_fund_list = []
    
    # Get all High School IDs
    id_number = get_school_ids()
    print(len(id_number))
    id_number = id_number[-10:-1]
    
    # Loop for Algebra scores
    for number in id_number:
        
        # URLs
        url_staar = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
                0&lev=C&id={number}&prgopt=reports%2Ftapr%2Fperformance.sas'''
            
        url_student=f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
                0&ccyy=2022&lev=C&id={number}&prgopt=reports/tapr/student.sas'''
            
        url_staff =f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug
                =0&ccyy=2022&lev=C&id={number}&prgopt=reports/tapr/staff.sas'''
            
        url_finance = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_service=appserv&_debug=
                0&_program=sfadhoc.new_Campus_actual21.sas&which_camp={number}'''
        
        # Ping URLs
        response_staar = get(url_staar)
            
        response_student = get(url_student)
           
        response_staff = get(url_staff)
            
        response_finance = get(url_finance)
        
        
        # Soupy soups
        soup_staar = BeautifulSoup(response_staar.content, 'html.parser')
        soup_student = BeautifulSoup(response_student.content, 'html.parser')
        soup_staff = BeautifulSoup(response_staff.content, 'html.parser')
        soup_finance = BeautifulSoup(response_finance.content, 'html.parser')
            
        try:
            
            # STAAR: Algebra score
            staar = soup_staar.find_all('tr')[20]
            algebra = staar.find_all('td')[2].text
            # Append
            algebra_list.append(algebra)
            
        except:
            algebra_list.append('Null')
        
        try:
        # Student Info: Economically disadvantaged, Emergent Bilingual, Emergent English Learner               
            ebel = soup_student.find_all('td', class_='r b data')[241].text
            econ = soup_student.find_all('td', class_='r b data')[217].text 
            econ_dis_list.append(econ)
            # Append
            ebel_list.append(ebel)
        
        except:
            ebel_list.append('Null')
        
        
        try:
        # Staff
            salary = soup_staff.find_all('td', class_='r b data')[-43].text
            masters = soup_staff.find_all('td', class_='r b data')[-114].text
            doct = soup_staff.find_all('td', class_='r b data')[-110].text
            beginning_teach = soup_staff.find_all('td', class_='r b data')[-106].text
            exp_5 = soup_staff.find_all('td', class_='r b data')[-102].text
            exp_10 = soup_staff.find_all('td', class_='r b data')[-98].text
            exp_1120 = soup_staff.find_all('td', class_='r b data')[-94].text
            exp_2130 = soup_staff.find_all('td', class_='r b data')[-90].text
            exp_plus = soup_staff.find_all('td', class_='r b data')[-86].text
            ratio = soup_staff.find_all('td', class_='r b data')[-83].text

            # Append
            salary_list.append(salary)
            masters_list.append(masters)
            doct_list.append(doct)
            beginning_teach_list.append(beginning_teach)
            exp_5_list.append(exp_5)
            exp_10_list.append(exp_10)
            exp_1120_list.append(exp_1120)
            exp_2130_list.append(exp_2130)
            exp_plus_list.append(exp_plus)
            ratio_list.append(ratio) 
            
        except:
            # Append
            salary_list.append(salary)
            masters_list.append(masters)
            doct_list.append(doct)
            beginning_teach_list.append(beginning_teach)
            exp_5_list.append(exp_5)
            exp_10_list.append(exp_10)
            exp_1120_list.append(exp_1120)
            exp_2130_list.append(exp_2130)
            exp_plus_list.append(exp_plus)
            ratio_list.append(ratio) 
        
        
         
        try:
            # Financial Info
            extra_fund = soup_finance.find_all('td', class_='r data')[91].text
            all_fund = soup_finance.find_all('td', class_='r data')[31].text

            extra_fund_list.append(extra_fund)
            all_fund_list.append(all_fund)

            # Append Lists
            id_number_list.append(cn)     
        
        except:
            # Append Lists
            id_number_list.append(cn) 
            
    
    df = pd.DataFrame({'id':id_number_list ,'algebra':algebra_list,
                      'eb_el': ebel_list,'econ_dis': econ_dis_list,
                      'salary': salary_list, 'masters': masters_list,
                      'doct': doct_list, 'beginning_teach': beginning_teach_list,
                      'exp_5': exp_5_list, 'exp_10': exp_10_list,
                      'exp_1120': exp_1120_list, 'exp_2130': exp_2130_list,
                      'exp_plus': exp_plus_list, 'ratio': ratio_list,
                      'extra_fund': extra_fund_list, 'all_fund': all_fund_list})
    print(algebra_list)
    print()
    print(ebel_list)
    print()
    print(econ_dis_list)
    print()
    print(salary_list)
    return df


In [20]:
lets_goooo = get_staar_students_staff_finance()
lets_goooo.head()

1784


UnboundLocalError: local variable 'salary' referenced before assignment

In [ ]:
lets_goooo

In [ ]:
def get_student_info():
    

In [ ]:
len(soup.find_all('tr')[20])

In [ ]:
def get_staar_ebel_econdis_1():
    staar_list = []
    ebel_list = []
    econ_dis_list = []
    id_number = get_school_ids()
    for cn in id_number:
        url = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
            0&lev=C&id={cn}&prgopt=reports%2Ftapr%2Fperformance.sas'''
        response = get(url)
        print(response) # print response to debug
        soup = BeautifulSoup(response.content, 'html.parser')
        rows = soup.find_all('tr')
        if len(rows) > 20:
            x = rows[20]
            algebra_cells = x.find_all('td')
            if len(algebra_cells) > 2:
                algebra = algebra_cells[2].text
                staar_list.append(algebra)
            else:
                print(f"Error: Not enough cells in algebra row for school {cn}")
            if len(algebra_cells) > 14:
                econ = algebra_cells[14].text
                econ_dis_list.append(econ)
            else:
                print(f"Error: Not enough cells in econ row for school {cn}")
            if len(algebra_cells) > 15:
                ebel = algebra_cells[15].text
                ebel_list.append(ebel)
            else:
                print(f"Error: Not enough cells in ebel row for school {cn}")
        else:
            print(f"Error: Not enough rows for school {cn}")
    df = pd.DataFrame({'id':id_number ,'algebra':staar_list,
                      'eb_el': ebel_list,'econ_dis': econ_dis_list})
    return df

In [ ]:
id_id = pd.read_csv('high_school_ids.csv', index_col=0)
firt_30 = list(id_id.id[1700:-1])

In [ ]:
firt_30

In [ ]:
staar_list = []
ebel_list = []
econ_dis_list = []
    
# Get all High School IDs
id_number = get_school_ids()
    
    
# Loop for Algebra scores
for cn in firt_30:
    
    try:
        url = f'''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
                0&lev=C&id={cn}&prgopt=reports%2Ftapr%2Fperformance.sas'''

        # Ping URL
        response = get(url)
        print(response)

        # Soupy soup
        soup = BeautifulSoup(response.content, 'html.parser')

        x = soup.find_all('tr')[20]

        # Algebra score
        algebra = x.find_all('td')[2].text

            # Econ Dis
        econ = x.find_all('td')[14].text

        # Eb El
        ebel = x.find_all('td')[15].text

        staar_list.append(algebra)
        econ_dis_list.append(econ)
        ebel_list.append(ebel)
        
    except:
        continue
    

In [ ]:
ebel_list

In [ ]:
url_1 = '''https://rptsvr1.tea.texas.gov/cgi/sas/broker?_service=marykay&_program=perfrept.perfmast.sas&_debug=
            0&lev=C&id=1902001&prgopt=reports%2Ftapr%2Fperformance.sas'''

# Ping URL
response = get(url)
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
soup

In [ ]:
x = soup.find_all('tr')[20]

## Load CSV

In [91]:
all_schools = pd.read_csv('school_data.csv', index_col=0)
all_schools.head()

,school_id,eng1,eng2,algebra,biology,history,ebel,econdis,salary,masters,doct,beginning_teach,ex_5,ex_10,ex_1120,ex_2130,ex_plus,extra_fund,all_fund,ratio
0,1902001,67%,82%,95%,88%,93%,1.2%,34.3%,"$55,259",22.0%,0.0%,0.0%,18.0%,16.4%,37.3%,22.5%,5.8%,"$1,852","$10,656",10.1
0,1903001,75%,87%,76%,92%,93%,1.3%,34.5%,"$48,689",29.4%,0.0%,0.0%,27.2%,21.6%,29.6%,10.8%,10.8%,"$2,056","$11,177",8.3
0,1904001,78%,80%,90%,87%,88%,4.7%,42.2%,"$51,538",37.0%,0.0%,5.0%,37.2%,15.2%,23.1%,11.4%,8.1%,"$2,151","$10,885",8.8
0,1906002,82%,80%,100%,100%,96%,2.9%,48.1%,"$58,500",9.8%,0.0%,0.0%,20.5%,20.4%,37.1%,11.0%,11.0%,"$1,899","$15,043",11.9
0,1907001,64%,66%,82%,83%,83%,13.8%,79.9%,"$52,703",20.8%,0.0%,3.5%,19.2%,17.4%,34.5%,11.7%,13.7%,"$1,504","$11,129",12.2


In [92]:
all_schools.shape

(1561, 20)

In [93]:
# Reset Index
all_schools = all_schools.reset_index()

In [94]:
# Drop duplicates
all_schools = all_schools.drop_duplicates()

In [95]:
# No dupes found
all_schools.shape

(1561, 21)

In [96]:
# Check for nulls
all_schools.isna().sum()

index              0
school_id          0
eng1               0
eng2               0
algebra            0
biology            0
history            0
ebel               0
econdis            0
salary             0
masters            0
doct               0
beginning_teach    0
ex_5               0
ex_10              0
ex_1120            0
ex_2130            0
ex_plus            0
extra_fund         0
all_fund           0
ratio              0
dtype: int64

In [97]:
# Info
all_schools.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1561 entries, 0 to 1560
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            1561 non-null   int64 
 1   school_id        1561 non-null   int64 
 2   eng1             1561 non-null   object
 3   eng2             1561 non-null   object
 4   algebra          1561 non-null   object
 5   biology          1561 non-null   object
 6   history          1561 non-null   object
 7   ebel             1561 non-null   object
 8   econdis          1561 non-null   object
 9   salary           1561 non-null   object
 10  masters          1561 non-null   object
 11  doct             1561 non-null   object
 12  beginning_teach  1561 non-null   object
 13  ex_5             1561 non-null   object
 14  ex_10            1561 non-null   object
 15  ex_1120          1561 non-null   object
 16  ex_2130          1561 non-null   object
 17  ex_plus          1561 non-null   

In [101]:
# Deep Copy of og df
all_schools_og = all_schools.copy()

## Cleaning Functions

In [141]:
loop_columns = [all_schools.columns]

def remove_percent_sign(df):
    
    loop_columns = list(df.columns[2:])
    
    for col in loop_columns:
        try:
            df[col] = df[col].str.replace('%','')
            
        except:
            continue
            
    return df


In [142]:
remove_percent_sign(all_schools)

,index,school_id,eng1,eng2,algebra,biology,history,ebel,econdis,salary,...,doct,beginning_teach,ex_5,ex_10,ex_1120,ex_2130,ex_plus,extra_fund,all_fund,ratio
0,0,1902001.0,67,82,95,88,93,1.2,34.3,"$55,259",...,0.0,0.0,18.0,16.4,37.3,22.5,5.8,"$1,852","$10,656",10.1
1,0,1903001.0,75,87,76,92,93,1.3,34.5,"$48,689",...,0.0,0.0,27.2,21.6,29.6,10.8,10.8,"$2,056","$11,177",8.3
2,0,1904001.0,78,80,90,87,88,4.7,42.2,"$51,538",...,0.0,5.0,37.2,15.2,23.1,11.4,8.1,"$2,151","$10,885",8.8
3,0,1906002.0,82,80,100,100,96,2.9,48.1,"$58,500",...,0.0,0.0,20.5,20.4,37.1,11.0,11.0,"$1,899","$15,043",11.9
4,0,1907001.0,64,66,82,83,83,13.8,79.9,"$52,703",...,0.0,3.5,19.2,17.4,34.5,11.7,13.7,"$1,504","$11,129",12.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,0,252901001.0,75,79,86,85,94,9.5,49.9,"$57,874",...,0.0,3.9,9.2,10.5,41.9,27.2,7.2,"$1,423","$9,852",13.1
1557,0,252903001.0,83,90,88,93,93,4.0,64.9,"$48,047",...,0.0,9.1,15.8,34.8,16.5,13.0,10.8,"$2,756","$13,508",7.7
1558,0,253901001.0,49,48,70,70,79,30.4,86.8,"$54,771",...,0.0,17.2,19.3,9.9,30.2,17.8,5.7,$113,"$7,916",13.9
1559,0,254901001.0,48,62,51,76,88,1.3,81.1,"$51,019",...,3.2,7.7,21.9,22.1,20.5,24.6,3.2,$60,"$7,617",16.9


In [133]:
list(all_schools.columns[2:])

['eng1',
 'eng2',
 'algebra',
 'biology',
 'history',
 'ebel',
 'econdis',
 'salary',
 'masters',
 'doct',
 'beginning_teach',
 'ex_5',
 'ex_10',
 'ex_1120',
 'ex_2130',
 'ex_plus',
 'extra_fund',
 'all_fund',
 'ratio']

In [149]:
def convert_dollars_to_float(x):

    fix = []
    blah = []
    something = [] #this list holds only the balance keys from the dictionary
    only_dollars = [] # this holds the values without the commas
    convert = []
    
    for d in x:
        d = d.replace(' ', '')
        fix.append(d)
        
    for e in fix:
        e = e.replace('-', '')
        blah.append(e)
        # this holds the converted values
        
    for a in blah:
        a = a.strip('$')
        something.append(a)
        
    for b in something:
        b = b.replace(",","")
        only_dollars.append(b)
        
    for c in only_dollars:
        c = float(c)
        convert.append(c)
    
        
    return convert

In [151]:
def change_dollars(df):
    df = df[df['salary'] != '-']
    df = df[df['salary'] != '?']
    df.salary = convert_dollars_to_float(df.salary)
    df.all_fund = convert_dollars_to_float(df.all_fund)
    df.extra_fund = convert_dollars_to_float(df.extra_fund)
    return df

In [157]:
def rename_cols(df):
    df = df.rename(columns= {'eng1': 'english_1',
                         'eng2': 'english_2',
                         'ebel': 'bilingual_or_english_learner',
                         'ex_5' : 'teacher_exp_5',
                         'ex_10': 'teacher_exp_6to10',
                         'ex_1120': 'teacher_exp_11to20',
                         'ex_2130': 'teacher_exp_21tp30',
                         'ex_plus' : 'teacher_exp_11plus',
                         'extra_fund': 'extracurricular_expend',
                         'all_fund': 'total_expend',
                         'ratio': 'student_teacher_ratio'
                         })
    return df

In [180]:
def remove_symbols(df):
    rows = df.columns.to_list()
    for row in rows:
        df = df[df[row] != '-         ']           
        df = df[df[row] != '*         ']
    return df

In [187]:
def clean_df(df):
    
    '''
    Cleans dataframe by replacing special characters with empty space
    and converting all columns to a numerical data type.
    '''
    
    # Covert dollar signs and special characters
    df = change_dollars(df)
    
    # Remove '*' and '-'
    df = remove_symbols(df)
    
    # Rename Columns
    df = rename_cols(df)
    
    # Columns to loop
    loop_columns = list(df.columns[2:])
    
    # Loop to remover '%' sign
    for col in loop_columns:
        try:
            df[col] = df[col].str.replace('%','').astype(float)
            
        except:
            continue
            
    return df

In [188]:
cleaned_df = clean_df(all_schools)

In [191]:
cleaned_df

,index,school_id,english_1,english_2,algebra,biology,history,bilingual_or_english_learner,econdis,salary,...,doct,beginning_teach,teacher_exp_5,teacher_exp_6to10,teacher_exp_11to20,teacher_exp_21tp30,teacher_exp_11plus,extracurricular_expend,total_expend,student_teacher_ratio
0,0,1902001.0,67.0,82.0,95.0,88.0,93.0,1.2,34.3,55259.0,...,0.0,0.0,18.0,16.4,37.3,22.5,5.8,1852.0,10656.0,10.1
1,0,1903001.0,75.0,87.0,76.0,92.0,93.0,1.3,34.5,48689.0,...,0.0,0.0,27.2,21.6,29.6,10.8,10.8,2056.0,11177.0,8.3
2,0,1904001.0,78.0,80.0,90.0,87.0,88.0,4.7,42.2,51538.0,...,0.0,5.0,37.2,15.2,23.1,11.4,8.1,2151.0,10885.0,8.8
3,0,1906002.0,82.0,80.0,100.0,100.0,96.0,2.9,48.1,58500.0,...,0.0,0.0,20.5,20.4,37.1,11.0,11.0,1899.0,15043.0,11.9
4,0,1907001.0,64.0,66.0,82.0,83.0,83.0,13.8,79.9,52703.0,...,0.0,3.5,19.2,17.4,34.5,11.7,13.7,1504.0,11129.0,12.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,0,252901001.0,75.0,79.0,86.0,85.0,94.0,9.5,49.9,57874.0,...,0.0,3.9,9.2,10.5,41.9,27.2,7.2,1423.0,9852.0,13.1
1557,0,252903001.0,83.0,90.0,88.0,93.0,93.0,4.0,64.9,48047.0,...,0.0,9.1,15.8,34.8,16.5,13.0,10.8,2756.0,13508.0,7.7
1558,0,253901001.0,49.0,48.0,70.0,70.0,79.0,30.4,86.8,54771.0,...,0.0,17.2,19.3,9.9,30.2,17.8,5.7,113.0,7916.0,13.9
1559,0,254901001.0,48.0,62.0,51.0,76.0,88.0,1.3,81.1,51019.0,...,3.2,7.7,21.9,22.1,20.5,24.6,3.2,60.0,7617.0,16.9


In [178]:

def remove_symbols_regex(df):
    columns = df.columns.to_list()
    for col in columns:
        try:
            
            df[col] = df[col].str.replace(r'/^a-zA-Z0-9 ]/g', '')
            
        except:
            continue
            
    return df

In [179]:
remove_symbols(all_schools).head(3)

/var/folders/86/j98_vzjs5dg0v5262tfx5yhw0000gn/T/ipykernel_5138/2433803100.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r'/^a-zA-Z0-9 ]/g', '')


,index,school_id,eng1,eng2,algebra,biology,history,ebel,econdis,salary,...,doct,beginning_teach,ex_5,ex_10,ex_1120,ex_2130,ex_plus,extra_fund,all_fund,ratio
0,0,1902001.0,67,82,95,88,93,1.2,34.3,"$55,259",...,0.0,0.0,18.0,16.4,37.3,22.5,5.8,"$1,852","$10,656",10.1
1,0,1903001.0,75,87,76,92,93,1.3,34.5,"$48,689",...,0.0,0.0,27.2,21.6,29.6,10.8,10.8,"$2,056","$11,177",8.3
2,0,1904001.0,78,80,90,87,88,4.7,42.2,"$51,538",...,0.0,5.0,37.2,15.2,23.1,11.4,8.1,"$2,151","$10,885",8.8


In [197]:
x = pd.read_csv('school_data.csv', index_col=0)
x.head()

,school_id,eng1,eng2,algebra,biology,history,ebel,econdis,salary,masters,doct,beginning_teach,ex_5,ex_10,ex_1120,ex_2130,ex_plus,extra_fund,all_fund,ratio
0,1902001,67%,82%,95%,88%,93%,1.2%,34.3%,"$55,259",22.0%,0.0%,0.0%,18.0%,16.4%,37.3%,22.5%,5.8%,"$1,852","$10,656",10.1
0,1903001,75%,87%,76%,92%,93%,1.3%,34.5%,"$48,689",29.4%,0.0%,0.0%,27.2%,21.6%,29.6%,10.8%,10.8%,"$2,056","$11,177",8.3
0,1904001,78%,80%,90%,87%,88%,4.7%,42.2%,"$51,538",37.0%,0.0%,5.0%,37.2%,15.2%,23.1%,11.4%,8.1%,"$2,151","$10,885",8.8
0,1906002,82%,80%,100%,100%,96%,2.9%,48.1%,"$58,500",9.8%,0.0%,0.0%,20.5%,20.4%,37.1%,11.0%,11.0%,"$1,899","$15,043",11.9
0,1907001,64%,66%,82%,83%,83%,13.8%,79.9%,"$52,703",20.8%,0.0%,3.5%,19.2%,17.4%,34.5%,11.7%,13.7%,"$1,504","$11,129",12.2


In [202]:
x.reset_index().drop('index', axis=1)

,school_id,eng1,eng2,algebra,biology,history,ebel,econdis,salary,masters,doct,beginning_teach,ex_5,ex_10,ex_1120,ex_2130,ex_plus,extra_fund,all_fund,ratio
0,1902001,67%,82%,95%,88%,93%,1.2%,34.3%,"$55,259",22.0%,0.0%,0.0%,18.0%,16.4%,37.3%,22.5%,5.8%,"$1,852","$10,656",10.1
1,1903001,75%,87%,76%,92%,93%,1.3%,34.5%,"$48,689",29.4%,0.0%,0.0%,27.2%,21.6%,29.6%,10.8%,10.8%,"$2,056","$11,177",8.3
2,1904001,78%,80%,90%,87%,88%,4.7%,42.2%,"$51,538",37.0%,0.0%,5.0%,37.2%,15.2%,23.1%,11.4%,8.1%,"$2,151","$10,885",8.8
3,1906002,82%,80%,100%,100%,96%,2.9%,48.1%,"$58,500",9.8%,0.0%,0.0%,20.5%,20.4%,37.1%,11.0%,11.0%,"$1,899","$15,043",11.9
4,1907001,64%,66%,82%,83%,83%,13.8%,79.9%,"$52,703",20.8%,0.0%,3.5%,19.2%,17.4%,34.5%,11.7%,13.7%,"$1,504","$11,129",12.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,252901001,75%,79%,86%,85%,94%,9.5%,49.9%,"$57,874",22.9%,0.0%,3.9%,9.2%,10.5%,41.9%,27.2%,7.2%,"$1,423","$9,852",13.1
1557,252903001,83%,90%,88%,93%,93%,4.0%,64.9%,"$48,047",27.9%,0.0%,9.1%,15.8%,34.8%,16.5%,13.0%,10.8%,"$2,756","$13,508",7.7
1558,253901001,49%,48%,70%,70%,79%,30.4%,86.8%,"$54,771",21.2%,0.0%,17.2%,19.3%,9.9%,30.2%,17.8%,5.7%,$113,"$7,916",13.9
1559,254901001,48%,62%,51%,76%,88%,1.3%,81.1%,"$51,019",17.4%,3.2%,7.7%,21.9%,22.1%,20.5%,24.6%,3.2%,$60,"$7,617",16.9


## Explore English STAAR scores

In [205]:
import prepare_copy as p

In [206]:
df = p.clean_df()
df.head()

,school_id,english_1,english_2,algebra,biology,history,bilingual_or_english_learner,econdis,salary,teacher_exp_6to10,extracurricular_expend,total_expend,student_teacher_ratio,teacher_exp_0to5,teacher_exp_11_plus,high_edu
0,1902001,67.0,82.0,95.0,88.0,93.0,1.2,34.3,55259.0,16.4,1852.0,10656.0,10.1,18.0,65.6,22.0
1,1903001,75.0,87.0,76.0,92.0,93.0,1.3,34.5,48689.0,21.6,2056.0,11177.0,8.3,27.2,51.2,29.4
2,1904001,78.0,80.0,90.0,87.0,88.0,4.7,42.2,51538.0,15.2,2151.0,10885.0,8.8,42.2,42.6,37.0
3,1906002,82.0,80.0,100.0,100.0,96.0,2.9,48.1,58500.0,20.4,1899.0,15043.0,11.9,20.5,59.1,9.8
4,1907001,64.0,66.0,82.0,83.0,83.0,13.8,79.9,52703.0,17.4,1504.0,11129.0,12.2,22.7,59.9,20.8


In [212]:
df.corr()

,school_id,english_1,english_2,algebra,biology,history,bilingual_or_english_learner,econdis,salary,teacher_exp_6to10,extracurricular_expend,total_expend,student_teacher_ratio,teacher_exp_0to5,teacher_exp_11_plus,high_edu
school_id,1.000000,0.005715,-0.003270,-0.004538,-0.002333,-0.012406,-0.014365,-0.052041,-0.059016,-0.042642,0.142962,0.058563,-0.141934,-0.084784,0.102443,-0.104058
english_1,0.005715,1.000000,0.832476,0.702677,0.721794,0.567064,-0.434795,-0.566804,0.030837,0.018751,0.147837,-0.159440,-0.056956,-0.286042,0.264944,0.137146
english_2,-0.003270,0.832476,1.000000,0.648409,0.817062,0.746659,-0.426149,-0.560583,0.079544,0.064513,0.161494,-0.217032,-0.002373,-0.277698,0.234203,0.141506
algebra,-0.004538,0.702677,0.648409,1.000000,0.643643,0.519039,-0.247730,-0.334292,-0.021095,0.000093,0.204670,-0.097419,-0.126999,-0.237602,0.227817,0.021673
biology,-0.002333,0.721794,0.817062,0.643643,1.000000,0.765080,-0.364884,-0.499743,0.054303,0.038545,0.199177,-0.187942,-0.052343,-0.254278,0.224617,0.073084
history,-0.012406,0.567064,0.746659,0.519039,0.765080,1.000000,-0.289671,-0.467822,0.116160,0.084718,0.127937,-0.204155,0.029041,-0.225146,0.173699,0.108696
bilingual_or_english_learner,-0.014365,-0.434795,-0.426149,-0.247730,-0.364884,-0.289671,1.000000,0.593040,0.181381,0.127291,-0.326251,-0.024918,0.191446,0.272408,-0.324388,0.044602
econdis,-0.052041,-0.566804,-0.560583,-0.334292,-0.499743,-0.467822,0.593040,1.000000,-0.048425,-0.043178,-0.226025,0.122795,0.072583,0.350291,-0.314335,-0.040042
salary,-0.059016,0.030837,0.079544,-0.021095,0.054303,0.116160,0.181381,-0.048425,1.000000,0.216543,-0.230121,-0.090416,0.223493,-0.247659,0.129969,0.382308
teacher_exp_6to10,-0.042642,0.018751,0.064513,0.000093,0.038545,0.084718,0.127291,-0.043178,0.216543,1.000000,-0.173858,-0.143909,0.208182,-0.175395,-0.329088,0.095818


In [213]:
# Copy of original
df_original = df.copy()

In [214]:
import explore as e

In [215]:
train, val, test = e.split_data(df)
train.shape, val.shape, test.shape

((778, 16), (334, 16), (279, 16))

In [220]:
# Correlations for History STAAR test
train.corr().sort_values('english_2').english_2

econdis                        -0.546219
bilingual_or_english_learner   -0.418480
teacher_exp_0to5               -0.320049
total_expend                   -0.253371
school_id                      -0.017341
student_teacher_ratio           0.017873
salary                          0.070578
teacher_exp_6to10               0.086055
high_edu                        0.126521
extracurricular_expend          0.148891
teacher_exp_11_plus             0.261092
algebra                         0.666678
history                         0.758639
biology                         0.820704
english_1                       0.840392
english_2                       1.000000
Name: english_2, dtype: float64

In [222]:
# Average English 2 score
df.english_2.mean()

71.66642703091301

In [237]:
# Loop for all averages
columns = list(df.columns[1:])

empty_list = []

for col in columns:
    
    x = df[col].mean()
    
    empty_list.append(x)

average = pd.DataFrame({'feature': list(df.columns[1:]), 'average': empty_list})
average

,feature,average
0,english_1,66.242272
1,english_2,71.666427
2,algebra,72.675054
3,biology,82.461538
4,history,87.777139
5,bilingual_or_english_learner,13.368440
6,econdis,59.362904
7,salary,57926.670740
8,teacher_exp_6to10,19.250611
9,extracurricular_expend,706.631919


**Questions**

1. On average, do schools with a higher than average economically disadvantaged lower STAAR scores?
2. Do schools with higher STAAR scores spend more per student on extra curriculars?
3. Of the schools with above average economically disadvantaged, does the school allocate less money toward extra curriculars?
4. Do schools who pay their teachers on average more than the state average have higher STAAR scores?


## Answer Q's

### 1. On average, do schools with a higher than average economically disadvantaged lower STAAR scores?